## This notebook is for running the Covid simulator and storing the output.

In [1]:
from time import time

import torch
from botorch.utils.transforms import unnormalize

from contextual_rs.test_functions.covid_exp_class import CovidSim, output_store, MAX_SEED, CovidSimV2, output_store_v2, eval_store_v2, CovidEvalV2

In [2]:
print(output_store)

/Users/saitcakmak/PycharmProjects/contextual_rs/contextual_rs/test_functions/covid_simulators/stored_simulations.pt


In [3]:
allocations = torch.tensor(
    [
        [0.2, 0.2],
        [0.2, 0.3],
        [0.2, 0.4],
        [0.2, 0.5],
        [0.2, 0.6],
        [0.3, 0.2],
        [0.3, 0.3],
        [0.3, 0.4],
        [0.3, 0.5],
        [0.4, 0.2],
        [0.4, 0.3],
        [0.4, 0.4],
        [0.5, 0.2],
        [0.5, 0.3],
        [0.6, 0.2],
    ]
)
prevalences = unnormalize(CovidSim.w_samples, CovidSim.bounds[:, 2:])

In [4]:
output_store

'/Users/saitcakmak/PycharmProjects/contextual_rs/contextual_rs/test_functions/covid_simulators/stored_simulations.pt'

In [5]:
verbose = False

try:
    stored_output = torch.load(output_store)
except FileNotFoundError:
    stored_output = dict()

sim = CovidSim()
start_time = time()
for x in allocations[:]:
    for y in prevalences:
        for seed in range(1, MAX_SEED + 1):
            X = torch.cat([x, y], dim=-1).view(1, 1, -1)
            time_elapsed = time() - start_time
            if verbose:
                print(f"X: {X}, seed: {seed}, time: {time_elapsed}")
            else:
                if len(stored_output) % 500 == 0:
                    print(".")
                elif len(stored_output) % 10 == 0:
                    print(".", end="")
            key = (tuple(X.flatten().tolist()), seed)
            if key in stored_output:
                continue
            stored_output[key] = sim(X, seed)
            torch.save(stored_output, output_store)


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [6]:
len(stored_output)

12150

In [7]:
X.shape

torch.Size([1, 1, 5])

In [8]:
stored_output[key]

tensor([[[14894.]]])

In [9]:
out_s = torch.load(output_store)

In [10]:
out_s[key]


tensor([[[14894.]]])

In [11]:
verbose = False

try:
    stored_output = torch.load(output_store_v2)
except FileNotFoundError:
    stored_output = dict()

sim = CovidSimV2()
start_time = time()
for x in allocations[:]:
    for y in sim.context_samples:
        for seed in range(1, MAX_SEED + 1):
            X = torch.cat([x, y], dim=-1).view(1, 1, -1)
            time_elapsed = time() - start_time
            if verbose:
                print(f"X: {X}, seed: {seed}, time: {time_elapsed}")
            else:
                if len(stored_output) % 500 == 0:
                    print(".")
                elif len(stored_output) % 10 == 0:
                    print(".", end="")
            key = (tuple(X.flatten().tolist()), seed)
            if key in stored_output:
                continue
            stored_output[key] = sim(X, seed)
            torch.save(stored_output, output_store_v2)


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Convert the V2 outputs to stored eval outputs

In [20]:
try:
    stored_evals = torch.load(eval_store_v2)
except FileNotFoundError:
    stored_evals = dict()

eval = CovidEvalV2()
start_time = time()
for x in allocations[:]:
    for y in CovidSimV2.context_samples:
        X = torch.cat([x, y], dim=-1).view(1, 1, -1)
        time_elapsed = time() - start_time
        if len(stored_output) % 100 == 0:
            print(".")
        elif len(stored_output) % 5 == 0:
            print(".", end="")
        key = tuple(X.flatten().tolist())
        if key in stored_evals:
            continue
        stored_evals[key] = eval(X, None)
        torch.save(stored_evals, eval_store_v2)


................................................................................................................................................................................................................................................